# Branch and Bound for the Quadratic Assignment Problem

Maxine: have a look at this thesis: `https://onedrive.live.com/?cid=801F87A75BCB7685&id=801F87A75BCB7685%21456&parId=801F87A75BCB7685%21283&o=OneUp`

In [1]:
##This is an initialization cell. Run this first
import pandas as pd
import numpy as np
from itertools import product
import time
import math
import matplotlib
import matplotlib.pyplot as plt

The functions below are for inputting the csv data (that is in the QAPLIB format) and making the matrices numpy arrays.

In [2]:
def CSVtoNumpyArray(rawdata):
    """
    Input: 
    rawdata = a csv file (insert name as a string)

    Output:
    two numpy matrices in a tuple
    """
    data = pd.read_csv(rawdata)  #Reads the data in as a pandas object
    c = data.columns
    column = int(c[0])
    final_data1 = data.iloc[:column,:].values  #Sets data into a series of numpy arrays of strings
    final_data2 = data.iloc[column:,:].values  #1 is for the first matrix(loc) and 2 is for the second(flow)
    

    #Forms the matrix as a numpy array (easier to work with) instead of an list of lists of strings
    def string_to_integers(final_data):
        matrix = []
        for j in range(column):
            string = final_data[j][0]
            string2 = string.split(" ")
            emptyarray = []
            for i in string2:
                if i != '':
                    emptyarray.append(int(i))
            matrix.append(emptyarray)
        npmatrix = np.array(matrix) 
        return npmatrix
    return string_to_integers(final_data1),string_to_integers(final_data2)

In [3]:
#REPOSITORY - can download from "Some instances from QAPLIB from the Github"

#small sized matrices(under 10x10) (quick on all methods)
matrix_size_4 = 'made4.csv'
matrix_size_5 = 'made5.csv'
matrix_size_6 = 'made6.csv'
matrix_size_7 = 'made7.csv'
matrix_size_8 = 'made8.csv'
matrix_size_9 = 'made9.csv'

#medium sized matrices(ranging from 10x10 to 30x30) (slow on deterministic methods, fast on heuristics)
matrix_size_10 = 'tai10a.csv'
matrix_size_11 = 'made11.csv'
matrix_size_12 = 'tai12a.csv'
matrix_size_15 = 'chr15a.csv' 
matrix_size_20 = 'chr20a.csv'
matrix_size_26 = 'bur26a.csv'

#large sized matrices(30x30 and bigger)(reasonably slow on the heuristics to a certain degree of accuracy)
matrix_size_40 = 'tai40a.csv'
matrix_size_60 = 'tai60.csv'
matrix_size_80 = 'tai80.csv'
matrix_size_256 = 'tai256c.csv'

datamatrix = CSVtoNumpyArray(matrix_size_4) # Decide the size of problem to run in the code (clue: 
                                                #the number in the original name is the size)
MatrixLoc = datamatrix[0]
MatrixFlow = datamatrix[1]

Below is the permutation generating function. For medium and large instances it would be wise to run this function to file separately first and then use the results for Exhaustive Search and/or Branch and Bound.

In [4]:
def QAPPermutations(iterable, r=None):
    """
    Input:
    String or numbers separated by a space
    optional= the length that the permutations must be
    
    Output:
    a generator of permutations
    
    Sourced from the itertools library and adjusted to suit this data type
    https://docs.python.org/3.4/library/itertools.html#itertools.permutations 
    """
    
    pool = iterable.split(" ")
    n = len(pool)
    r = n if r is None else r
    for indices in product(range(n), repeat=r):
        if len(set(indices)) == r:
            yield list(pool[i] for i in indices)

In [5]:
def FitnessFunction(MatrixLoc, MatrixFlow, perm):
    """
    Input:
    MatrixLoc: numpy array
    MatrixFlow: numpy array
    perm: numpy array
    
    Output:
    total: int

    """
    #initialization
    length = len(MatrixLoc)
    total = 0
    
    #the sum
    for i in range(length):
        for j in range(length):
            if i!=j:
                total += MatrixLoc[i][j]*MatrixFlow[int(perm[i])][int(perm[j])]
    return total         

In [14]:
def BranchandBound(MatrixLoc,MatrixFlow):
    """
    Input:
    MatrixLoc
    MatrixFlow
    
    Output:
    The optimal permutation
    the optimal cost
    in a tuple
    """
    #initialization
    start = time.time()
    matrix_length = len(MatrixLoc)
    arraysol = []
    bettersol = math.inf
    bettersolind = 0
    thestring = ""
    
    #Generate the permutations
    for i in range(0,matrix_length):  #this is making a string of numbers from 0 to the size of the matrix -1
        thestring += str(i) + " "
    thestring = thestring[:-1]
    listofpermutations = list(QAPPermutations(thestring,matrix_length))
    no_of_permutations = len(listofpermutations)
    
    #Generate the multiples (that function we are optimising)
    for k in range(no_of_permutations):
        perm = listofpermutations[k]
        total = 0
        i =0
        while i < matrix_length and bettersol>=total:
            j=0
            while j<matrix_length:
                if i!=j:
                    total += MatrixLoc[i][j]*MatrixFlow[int(perm[i])][int(perm[j])]
                j += 1
            i += 1
        if bettersol>total:
            bettersol = total
            bettersolind = k
        
    
    finalcost = bettersol
    finalindex = bettersolind
    end = time.time()
    thetime = end - start
    
    return listofpermutations[finalindex],finalcost,thetime

In [15]:
BranchandBound(MatrixLoc, MatrixFlow)

(['2', '3', '0', '1'], 790, 0.003996849060058594)